# Experiment 1 - Predict using XGBoost

### Notebook 1: Organizing the dataset

**Details**.
* O dataset de teste não tem os labels, então:
    * Incluir uma coluna dizendo a qual família de motores aqueles dados fazem parte
    * Concatenar train-label de todos os train
    * Ordenar por timestamp
    * Dividir em treino - teste simples

In [14]:
from pathlib import Path
import pandas as pd

In [15]:
def load_data(data_path):   
    operational_settings = ['op_setting_{}'.format(i + 1) for i in range (3)]
    sensor_columns = ['sensor_meas_{}'.format(i + 1) for i in range(26)]
    cols = ['engine_no', 'time_in_cycles'] + operational_settings + sensor_columns
    
    data = pd.read_csv(data_path, sep=' ', header=-1, names=cols)
    data = data.drop(cols[-5:], axis=1)
    data['index'] = data.index
    data.index = data['index']
    data['time'] = pd.date_range('1/1/2000', periods=data.shape[0], freq='600s')
#     print('Loaded data with:\n{} Recordings\n{} Engines'.format(
#         data.shape[0], len(data['engine_no'].unique())))
#     print('21 Sensor Measurements\n3 Operational Settings')
    return data

In [16]:
def load_target(data_path):
    cols = ['RUL']
    data = pd.read_csv(data_path, sep=' ', header=-1, names=cols, index_col=False)
    return data

In [17]:
# Load dos trains e targets
PATH   = Path('../data/CMAPSSData')
train01 = load_data(PATH/'train_FD001.txt')
target01 = load_target(PATH/'RUL_FD001.txt')

train02 = load_data(PATH/'train_FD002.txt')
target02 = load_target(PATH/'RUL_FD002.txt')

train03 = load_data(PATH/'train_FD003.txt')
target03 = load_target(PATH/'RUL_FD003.txt')

train04 = load_data(PATH/'train_FD004.txt')
target04 = load_target(PATH/'RUL_FD004.txt')

In [18]:
# incluir a coluna com a família do motor
train01['engine_family'] = 1
train02['engine_family'] = 2
train03['engine_family'] = 3
train04['engine_family'] = 4

In [19]:
# concatenar train-label 
data01 = pd.concat([train01, target01], axis=1, sort=False)
data02 = pd.concat([train02, target02], axis=1, sort=False)
data03 = pd.concat([train03, target03], axis=1, sort=False)
data04 = pd.concat([train04, target04], axis=1, sort=False)

In [20]:
data_all = pd.concat([data01, data02, data03, data04], axis = 0, sort=False)

In [23]:
# Ordenar por timestamp
data = data.sort_values(by=['time'])

In [24]:
print(data01.shape)
print(data02.shape)
print(data03.shape)
print(data04.shape)
print(data_all.shape)

(20631, 30)
(53759, 30)
(24720, 30)
(61249, 30)
(160359, 30)


In [25]:
# trocar time por datetime
time = pd.to_datetime(data['time'])
data.drop(['time'], axis=1, inplace=True)
data = pd.concat([data, time], axis=1)

In [26]:
# trocar int por float
for col in ['engine_no', 'time_in_cycles', 'sensor_meas_17','sensor_meas_18', 'engine_family']:
    ints = data[col].astype('float64')
    data.drop([col], axis=1, inplace=True)
    data = pd.concat([data, ints], axis=1)

In [27]:
# persist new dataframe
data.to_csv('../data/CMAPSSData/data_all.csv')

end =================================================================